In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
df = pd.read_csv('../data/Laserpowervsinjcurrent.csv')
#read all columns data into variables
current = np.array(df['I (mA)'])
current_error = np.array(df['delta I'])
unatt_pow = np.array(df['Pnoatt [muW]'])
unatt_pow_error = np.array(df['delta Pnoatt'])
att_pow = np.array(df['Patt [muW]'])
atta_pow_error = np.array(df['delta Patt'])